In [4]:
import numpy as np
import pandas as pd
from feature_engine import discretisation as dsc
from feature_engine import encoding as ce
from feature_engine import outliers as outr
from feature_engine import transformation as vt
from feature_engine.imputation import MeanMedianImputer
from sklearn.model_selection import train_test_split



In [9]:
def load_titanic():
    data = pd.read_csv("https://www.openml.org/data/get_csv/16826755/phpMYEkMl")
    data = data.replace("?", np.nan)
    data["cabin"] = data["cabin"].astype(str).str[0]
    data["pclass"] = data["pclass"].astype("O")
    data["embarked"].fillna("C", inplace=True)
    return data


# Load dataset
def load_titanic2():
    data = pd.read_csv("https://www.openml.org/data/get_csv/16826755/phpMYEkMl")
    data = data.replace("?", np.nan)
    data["cabin"] = data["cabin"].astype(str).str[0]
    data["pclass"] = data["pclass"].astype("O")
    data["embarked"].fillna("C", inplace=True)
    data["fare"] = data["fare"].astype("float")
    data["fare"].fillna(data["fare"].median(), inplace=True)
    data["age"] = data["age"].astype("float")
    data["age"].fillna(data["age"].median(), inplace=True)
    return data

In [25]:
# Load dataset
data = pd.read_csv("creditApprovalUCI.csv")

data.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,202.0,0,1
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,43.0,560,1
2,a,24.50,NaN,u,g,q,h,NaN,NaN,NaN,0,f,g,280.0,824,1
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,100.0,3,1
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,120.0,0,1


In [26]:
# Separate into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
   data.drop("A16", axis=1), data["A16"], test_size=0.3, random_state=0)

# numerical variables with missing data
X_train[['A8', 'A11']].isnull().median()

A8     0.0
A11    0.0
dtype: float64

In [27]:
# Set up the imputer
median_imputer = MeanMedianImputer(
   imputation_method="median", variables=["A2", "A3", "A8", "A11", "A15"])
# fit the imputer
median_imputer.fit(X_train)

median_imputer.imputer_dict_

{'A2': 28.835, 'A3': 2.75, 'A8': 1.0, 'A11': 0.0, 'A15': 6.0}

In [5]:


# Load dataset
data = pd.read_csv("creditApprovalUCI.csv")

# Separate into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
   data.drop("A16", axis=1), data["A16"], test_size=0.3, random_state=0)

# Set up the imputer
median_imputer = MeanMedianImputer(
   imputation_method="median", variables=["A2", "A3", "A8", "A11", "A15"])
# fit the imputer
median_imputer.fit(X_train)

# transform the data
X_train = median_imputer.transform(X_train)
X_test = median_imputer.transform(X_test)

In [7]:
X_train.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15
596,a,46.08,3.000,u,g,c,v,2.375,t,t,8,t,g,396.0,4159
303,a,15.92,2.875,u,g,q,v,0.085,f,f,0,f,g,120.0,0
204,b,36.33,2.125,y,p,w,v,0.085,t,t,1,f,g,50.0,1187
351,b,22.17,0.585,y,p,ff,ff,0.000,f,f,0,f,g,100.0,0
118,b,57.83,7.040,u,g,m,v,14.000,t,t,6,t,g,360.0,1332


In [10]:
from feature_engine import encoding as ce

# Load dataset
data = load_titanic()

# Separate into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
  data.drop(["survived", "name", "ticket"], axis=1), data["survived"], test_size=0.3, random_state=0,)

# set up the encoder
encoder = ce.CountFrequencyEncoder(
    encoding_method="frequency", variables=["cabin", "pclass", "embarked"])

# fit the encoder
encoder.fit(X_train)

# transform the data
train_t = encoder.transform(X_train)
test_t = encoder.transform(X_test)

In [13]:
from feature_engine import transformation as vt

# Load dataset
data = pd.read_csv("houseprice.csv")

# Separate into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    data.drop(["Id", "SalePrice"], axis=1), data["SalePrice"], test_size=0.3, random_state=0,)

# set up the variable transformer
tf = vt.BoxCoxTransformer(variables=["LotArea", "GrLivArea"])

# fit the transformer
tf.fit(X_train)

# transform the data
train_t = tf.transform(X_train)
test_t = tf.transform(X_test)


In [14]:
from feature_engine import discretisation as dsc

# Load dataset
data = data = pd.read_csv("houseprice.csv")

# Separate into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    data.drop(["Id", "SalePrice"], axis=1), data["SalePrice"], test_size=0.3, random_state=0,)

# set up the discretisation transformer
disc = dsc.DecisionTreeDiscretiser(
    cv=3, scoring="neg_mean_squared_error", variables=["LotArea", "GrLivArea"], regression=True,)

# fit the transformer
disc.fit(X_train, y_train)

# transform the data
train_t = disc.transform(X_train)
test_t = disc.transform(X_test)


In [16]:
from feature_engine import outliers as outr

# Load dataset
data = load_titanic2()

# Separate into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    data.drop(["survived", "name", "ticket"], axis=1), data["survived"], test_size=0.3, random_state=0,)

# set up the capper
capper = outr.Winsorizer(
    capping_method="gaussian", tail="right", fold=3, variables=["age", "fare"])

# fit the capper
capper.fit(X_train)

# transform the data
train_t = capper.transform(X_train)
test_t = capper.transform(X_test)